In [96]:
import itertools

In [141]:
## strip out extra lines between entries
## todo: convert to regex

ft = open("tables.txt", 'r', encoding="utf8")
ft_new = open("tables_clean.txt", 'w', encoding="utf8")

line1 = ft.readline()
line2 = ft.readline()
isempty1 = line1.isspace()
isempty2 = line2.isspace()

ft_new.write(line1)
for line3 in ft:
    isempty3 = line3.isspace()
    if (not isempty2) or isempty1 or isempty3:
        ft_new.write(line2)
    
    line1 = line2
    line2 = line3
    isempty1 = isempty2
    isempty2 = isempty3
    
ft_new.write(line3)

ft.close()
ft_new.close()

In [142]:
# ## save each entry as separate text file
# ft = open("tables_clean.txt", 'r')
# l = []

    

In [143]:
# ## strip out quotations and save to new file
# ft = open("tables_clean.txt", 'r', encoding='utf8')
# ft_new = open("tables_noquotes.txt", 'w', encoding='utf8')
# ft_quotes = open("quotations.txt", 'w')

# line1 = ft.readline()

# for line2 in ft:

#     if "\u2014" in line2[0]: # \u2014 is em dash
# #         print(line1, line2)
#         ft_quotes.writelines([line1, line2, "\n"])
#         line1 = ft.readline()
#     else:  
#         ft_new.write(line1)
#         line1 = line2;

# ft.close()
# ft_new.close()
# ft_quotes.close()

In [144]:
## input filename, output dictionary of {flavor: [text as list of lines]} pairs
ft = open("tables_clean.txt", 'r', encoding='utf8')

lines = ft.readlines()

flavor = lines[0].lower()
text = []
flavor_dict = {}

for i in range(1, len(lines)-1):
    cur = lines[i].rstrip()
    
    if cur.isupper() and lines[i-1].isspace() and lines[i+1].isspace():
        flavor_dict[flavor] = text
        flavor = cur.lower()
        text = []
    else:
        text.append(cur)
        
ft.close()

In [145]:
## pull out quotations and recommended dishes and put in separate dictionary
## can we distinguish "dishes" from other quotations?
quotation_dict = {}
for flavor in flavor_dict:
    lines = flavor_dict[flavor]
    q = []
    ftext = []
    quotation_dict[flavor] = {'author': [], 'quotation': []}
    for l in lines:
        if not l:
            if q and "Dishes" not in q: ftext.append(q)
            q = []
        elif "\u2014" in l[0]:
            quotation_dict[flavor]['quotation'].append(q)
            quotation_dict[flavor]['author'].append(l[1:])
            q = []
        else:
            q.append(l)
    flavor_dict[flavor] = ftext

In [146]:
## strip out Flavor Affinities and put in separate dictionary
affinity_dict = {}
for flavor in flavor_dict:
    lines = list(itertools.chain.from_iterable(flavor_dict[flavor]))
#     lines = flavor_dict[flavor]
    flaf = []
    ftext = []
    for l in lines:
        if "+" in l:
            flaf.append(l)
        elif "Flavor Affinit" not in l:
            ftext.append(l)
    flavor_dict[flavor] = ftext
    affinity_dict[flavor] = flaf

In [147]:
## strip out "Avoid" and put in separate dictionary
avoid_dict = {}
for flavor in flavor_dict:
    ftext = []
    avoid = []
    is_avoid = False
    for l in flavor_dict[flavor]:
        if is_avoid:
            avoid.append(l)
        elif "AVOID" in l:
            is_avoid = True
        else:
            ftext.append(l)
    flavor_dict[flavor] = ftext
    avoid_dict[flavor] = avoid

In [148]:
## strip out Pairings and put in separate dictionary
pairing_dict = {}
keywords = ["Season:", "Taste:", "Function:", "Weight:", "Volume:", "Techniques:", "Tips:", "Techniques/Tips:"] 
for flavor in flavor_dict:
    plist = []
    ftext = []
    for l in flavor_dict[flavor]:
        if any(word in keywords for word in l.split()):
            ftext.append(l)
        else:
            plist.append(l)
    pairing_dict[flavor] = plist
    flavor_dict[flavor] = ftext

In [149]:
print(quotation_dict['apples']['author'][7], "\n", quotation_dict['apples']['quotation'][7])

JOSÉ ANDRÉS, CAFÉ ATLÁNTICO (WASHINGTON, DC) 
 ['I have always been very fond of chef Frédy Girardet [who earned three Michelin stars at his restaurant in Switzerland before retiring in 1996]. When I was young, I cooked almost every recipe from his book and visited his restaurant. One of his most interesting desserts was an apple dessert made of apples in the shape of little balls. The dessert broke away from cooking apples whole in the traditional way, which alone inspired me. He cooked them over a very high heat for two minutes and put them into a red wine reduction that had cinnamon, orange peel, and sugar. This was put onto a sheet tray that needed to be shaken for an hour so the apples would not dry out. The apples would absorb these flavors like a sponge and would then be served with vanilla ice cream.', 'In that spirit, we transformed this dessert. We transformed the wine by “espherication” so that it creates a bubble of liquid that explodes in your mouth. We cut apples with a m

In [150]:
avoid_dict['indian cuisine']

['beef, for religious reasons, say some',
 'pork, for religious reasons, say some']

In [151]:
affinity_dict['apples']

['apples + almonds + caramel',
 'apples + almonds + Armagnac + crème fraîche + raisins',
 'apples + apricots + pine nuts + rosemary',
 'apples + brown sugar + cream + walnuts',
 'apples + Calvados + cranberries + maple syrup',
 'apples + caramel + cinnamon',
 'apples + caramel + cinnamon + dates + lemon confit + quince + ras el hanout + vanilla',
 'apples + caramel + peanuts',
 'apples + caramel + pecans',
 'apples + caramel + pistachios + vanilla',
 'apples + celery + walnuts',
 'apples + cinnamon + cranberries',
 'apples + cinnamon + dark chocolate + yams',
 'apples + cream + ginger',
 'apples + ginger + hazelnuts',
 'apples + ginger + lemon + quince + sugar',
 'apples + honey + lemon thyme',
 'apples + raisins + rum',
 'apples + red cabbage + cinnamon']

In [152]:
pairing_dict['apples']

['allspice',
 'almonds',
 'apple cider or juice',
 'applejack',
 'apricots: dried, jam, puree',
 'Armagnac',
 'bacon',
 'bay leaf',
 'beef',
 'blackberries',
 'bourbon',
 'brandy, esp. apple',
 'brioche',
 'BUTTER, UNSALTED',
 'butterscotch',
 'cabbage, red',
 'CALVADOS',
 'CARAMEL',
 'cardamom',
 'celery',
 'celery root',
 'cheese: Camembert, cheddar, goat, Gruyère',
 'cherries: dried, fresh',
 'chestnuts',
 'chicken',
 'chives',
 'cider',
 '*CINNAMON',
 'cloves',
 'cognac',
 'Cointreau',
 'coriander',
 'cranberries',
 'CREAM AND ICE CREAM',
 'crème anglaise (sauce)',
 'crème fraîche',
 'crust: pastry, pie',
 'cumin',
 'currants, esp. black, and currant jelly',
 'curry powder',
 'custards',
 'dates',
 'duck',
 'eggplant',
 'fennel',
 'French cuisine, esp. from Normandy',
 'frisée',
 'ginger',
 'goose',
 'hazelnuts',
 'honey, esp. chestnut, wildflower',
 'horseradish',
 'ice cream',
 'Kirsch',
 'lavender',
 'LEMON: JUICE, ZEST',
 'lemon thyme',
 'lychees',
 'Madeira',
 'maple syrup',
 

In [153]:
flavor_dict['apples']

['Season: autumn',
 'Taste: sweet, astringent',
 'Function: cooling',
 'Weight: medium',
 'Volume: quiet–moderate',
 'Techniques: bake, caramelize, deep-fry (e.g., as fritters), grill, poach, raw, sauté, stew']